In [1]:
%load_ext autoreload
%autoreload 2

from amber.adapters.text_snippet_dataset import TextSnippetDataset
from amber.core.language_model import LanguageModel
from pathlib import Path

from playground.train_sae import STORE_DIR

MODEL_ID = "sshleifer/tiny-gpt2"  # tiny model for quick experimentation
HF_DATASET = "roneneldan/TinyStories"

CACHE_DIR = Path("./store/tinystories")
STORE_DIR = Path("./store/tiny-gpt2")
DEVICE = 'cpu'
DATA_SPLIT = "train"
TEXT_FIELD = "text"
DATA_LIMIT = 200  # keep small for a quick demo
MAX_LENGTH = 128

model = LanguageModel.from_huggingface(MODEL_ID)
model.model.to(DEVICE)

dataset = TextSnippetDataset.from_huggingface(
    HF_DATASET,
    split=DATA_SPLIT,
    cache_dir=str(CACHE_DIR),
    text_field=TEXT_FIELD,
    limit=DATA_LIMIT,
)

/Users/adam/Projects/Inzynierka/codebase/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since roneneldan/TinyStories couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/adam/.cache/huggingface/datasets/roneneldan___tiny_stories/default/0.0.0/f54c09fd23315a6f9c86f9dc80f725de7d8f9c64 (last modified on Sun Sep 28 23:39:19 2025).
Saving the dataset (1/1 shards): 100%|██████████| 200/200 [00:00<00:00, 95694.82 examples/s] 


In [2]:
import os
from amber.mechanistic.autoencoder.autoencoder import Autoencoder

sae_models = STORE_DIR / "sae_models"
last_run = os.listdir(sae_models)[-1]

sae_layer = Autoencoder.load_model(sae_models / last_run / 'final.pt')[0]
sae_layer.metadata

2025-10-03 18:38:20,668 [INFO] amber.mechanistic.autoencoder.autoencoder: 
Loaded model from store/tiny-gpt2/sae_models/tinystories_20250930_203105/final.pt
n_latents=2, n_inputs=2, activation=TopK_4, tied=False


{'run_name': 'tinystories_20250930_203105',
 'model': 'GPT2LMHeadModel',
 'layer_signature': 'gpt2lmheadmodel_transformer_h_1_ln_2',
 'dataset': {'cache_dir': 'store/tinystories', 'length': 200},
 'options': {'dtype': None,
  'max_length': 128,
  'save_inputs': True,
  'batch_size': 8}}

In [4]:
model_name = sae_layer.metadata['model']
layer_signature = sae_layer.metadata['layer_signature']


model = LanguageModel.from_huggingface(
    "sshleifer/tiny-gpt2",
)

In [5]:
model.layers.register_new_layer('sae', sae_layer, layer_signature)

In [ ]:
model